#. DATA정보정리
pclass : passenger Class 승객등급
survuved : 생존여부 (생존1, 사망0)
name : 이름
sex : 성별
sibsp : 동승한 형제 또는 배우자 수
parch : 동승한 보모 또는 자녀 수
ticket : 티켓 번호
fare : 승객 지불 요금
cabin : 선실 이름
embarked : 승선항(c = 쉘부르크, Q= 퀸즈타운, S=사우스 햄튼)
body : 사망자 확인 번호
bome.dest : 고향/목적지

In [40]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier

In [42]:
cat_types={}

def process_data(filename):
    """
    first run sets cat_types
    """
    dd=pd.read_csv(filename).set_index("PassengerId")
    
    dd["Ticket2"]=dd["Ticket"].str[:2]
    dd["Cabin2"]=dd["Cabin"].str[:2]
    dd["Ticket_nums"]=dd["Ticket"].str.extract("([0-9]{3,})").astype(float)
    
    for cat_col in ["Sex", "Embarked", "Ticket2", "Cabin2"]:
        if cat_col not in cat_types:
            dd[cat_col]=dd[cat_col].astype("category")
            cat_types[cat_col]=dd[cat_col].cat.categories
        else:
            dd[cat_col]=dd[cat_col].astype("category").cat.set_categories(cat_types[cat_col])
    
    return dd

D=process_data("input/train.csv")
D_test=process_data("input/test.csv")
D

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket2,Cabin2,Ticket_nums
PassengerId,,,,,,,,,,,,,,
0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S,20,C1,209245.0
1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S,27,NaN,27323.0
2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S,CA,NaN,457703.0
3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S,A.,NaN,10866.0
4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S,42,NaN,427635.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2,"Bell, Adele",female,62.00,0,0,PC 15008,14.86,D17243,C,PC,D1,15008.0
99996,0,2,"Brown, Herman",male,66.00,0,0,13273,11.15,NaN,S,13,NaN,13273.0
99997,0,3,"Childress, Charles",male,37.00,0,0,NaN,9.95,NaN,S,NaN,NaN,NaN


In [51]:
feats=["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Fare", "Ticket2", "Cabin2", "Ticket_nums"]

X=D[feats]
y=D["Survived"]

model = LGBMClassifier(
    learning_rate=0.03,
    num_leaves=20,
    n_estimators=300,
)

model.fit(X, y);

In [52]:
X_test = D_test[feats]

pred = model.predict(X_test)

submission=pd.DataFrame({"PassengerId": X_test.index, "Survived": pred})
submission.to_csv("input/submission_baseline.csv", index=False)
submission.head()

,PassengerId,Survived
0,100000,0
1,100001,0
2,100002,1
3,100003,0
4,100004,1
